In [0]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [0]:
train_data = dsets.MNIST(root='./Data',train=True,transform=transforms.transforms.ToTensor(),download=True)

In [0]:
test_data = dsets.MNIST(root='./Data',train=False,transform=transforms.transforms.ToTensor(),download=True)

In [0]:
### Making Train and Test Data Iteratable

In [0]:
train_iter = torch.utils.data.DataLoader(dataset=train_data,batch_size=100,shuffle=True)
test_iter = torch.utils.data.DataLoader(dataset=test_data,batch_size=100,shuffle=False)

In [0]:
class FeedForwardNN(nn.Module):
 
  def __init__(self,input_dim,hidden_dim,output_dim):
    
    super(FeedForwardNN,self).__init__()
    self.fc1 = nn.Linear(input_dim,hidden_dim)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_dim,input_dim)
    
  def foward(self,X):
    
    out = self.fc1(X)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [0]:
input_dim = 28 * 28
hidden_dim = 100
output_dim =10

In [0]:
ffnn = FeedForwardNN(input_dim,hidden_dim,output_dim)

In [75]:
if torch.cuda.is_available():
  
  print("GPU Available")
  ffnn.cuda()

GPU Available


In [0]:
loss = nn.CrossEntropyLoss()

In [0]:
optim = torch.optim.SGD(ffnn.parameters(),lr=0.001)

In [0]:
no_of_epocs = 10

In [79]:
for epoc in range(no_of_epocs):
  iter = 0
  print("Epoc:",epoc)
  for i,(images,lables) in enumerate(train_iter):
    
    images = Variable(images.view(-1,28*28).cuda())
    lables = Variable(lables.cuda())
    optim.zero_grad()
    output = ffnn.foward(images)
    c_loss = loss(output,lables)
    c_loss.backward()
    optim.step()
    iter += 1
    
    if iter % 100 == 0 :
      
      total = 0
      correct = 0
      
      for images,lables in test_iter:
        
        images = Variable(images.view(-1,28*28).cuda())
        lables = Variable(lables.cuda())
        output = ffnn.foward(images)
        _,pred = torch.max(output.data,1)
        total += lables.size(0)
        correct += (pred.cpu() == lables.cpu()).sum()
        
      acc = (correct.numpy()/total) * 100
      print("Iter:{},Loss:{},Accuracy:{}".format(iter,c_loss.data,acc))

Epoc: 0
Iter:100,Loss:6.56461238861084,Accuracy:2.41
Iter:200,Loss:6.4794769287109375,Accuracy:10.51
Iter:300,Loss:6.340563774108887,Accuracy:17.49
Iter:400,Loss:6.191898822784424,Accuracy:22.470000000000002
Iter:500,Loss:5.996387958526611,Accuracy:25.56
Iter:600,Loss:5.736166477203369,Accuracy:28.499999999999996
Epoc: 1
Iter:100,Loss:5.4073381423950195,Accuracy:29.14
Iter:200,Loss:4.981744766235352,Accuracy:29.34
Iter:300,Loss:4.406479358673096,Accuracy:29.28
Iter:400,Loss:3.75350284576416,Accuracy:31.3
Iter:500,Loss:3.0389299392700195,Accuracy:34.03
Iter:600,Loss:2.843745231628418,Accuracy:40.47
Epoc: 2
Iter:100,Loss:2.5263075828552246,Accuracy:52.059999999999995
Iter:200,Loss:2.353151321411133,Accuracy:62.45
Iter:300,Loss:2.275651216506958,Accuracy:67.72
Iter:400,Loss:2.2014389038085938,Accuracy:70.28
Iter:500,Loss:2.0620243549346924,Accuracy:70.30999999999999
Iter:600,Loss:1.9927846193313599,Accuracy:71.64
Epoc: 3
Iter:100,Loss:1.946397304534912,Accuracy:72.83
Iter:200,Loss:1.86478